In [142]:
%run 'preprocess.ipynb'

(64667, 381)
(64667, 366)
(16000, 366)


In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.externals import joblib

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from scipy.stats import randint as sp_randint

In [122]:
forest = RandomForestClassifier(n_jobs=-1)
grad = GradientBoostingClassifier(n_estimators=50)

In [105]:
eclf_grid = VotingClassifier(estimators=[('forest', forest), ('grad', grad)], voting='soft')

params = {"forest__max_depth": sp_randint(1, 366),
              "forest__max_features": sp_randint(1, 366),
              "forest__min_samples_split": sp_randint(1, 500),
              "forest__criterion": ["gini", "entropy"],
              "grad__max_depth": sp_randint(2, 10),
              "grad__min_samples_split": sp_randint(1, 500),
              "grad__max_features": sp_randint(1, 366)
         }

grid_result = RandomizedSearchCV(eclf_grid, n_iter=50, cv=4, param_distributions=params, verbose=2, n_jobs=-1)

grid_result.fit(X, y)

Fitting 4 folds for each of 50 candidates, totalling 200 fits
[CV] grad__max_features=48, forest__max_features=158, forest__criterion=entropy, forest__max_depth=24, grad__max_depth=4, grad__min_samples_split=210, forest__min_samples_split=239 
[CV] grad__max_features=48, forest__max_features=158, forest__criterion=entropy, forest__max_depth=24, grad__max_depth=4, grad__min_samples_split=210, forest__min_samples_split=239 
[CV] grad__max_features=48, forest__max_features=158, forest__criterion=entropy, forest__max_depth=24, grad__max_depth=4, grad__min_samples_split=210, forest__min_samples_split=239 
[CV] grad__max_features=48, forest__max_features=158, forest__criterion=entropy, forest__max_depth=24, grad__max_depth=4, grad__min_samples_split=210, forest__min_samples_split=239 
[CV] grad__max_features=306, forest__max_features=65, forest__criterion=entropy, forest__max_depth=365, grad__max_depth=7, grad__min_samples_split=306, forest__min_samples_split=309 
[CV] grad__max_features=306

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 21.8min


[CV]  grad__max_features=55, forest__max_features=249, forest__criterion=entropy, forest__max_depth=16, grad__max_depth=8, grad__min_samples_split=28, forest__min_samples_split=211, total= 2.9min
[CV] grad__max_features=216, forest__max_features=199, forest__criterion=entropy, forest__max_depth=217, grad__max_depth=6, grad__min_samples_split=93, forest__min_samples_split=167 
[CV]  grad__max_features=55, forest__max_features=249, forest__criterion=entropy, forest__max_depth=16, grad__max_depth=8, grad__min_samples_split=28, forest__min_samples_split=211, total= 2.9min
[CV] grad__max_features=216, forest__max_features=199, forest__criterion=entropy, forest__max_depth=217, grad__max_depth=6, grad__min_samples_split=93, forest__min_samples_split=167 
[CV]  grad__max_features=55, forest__max_features=249, forest__criterion=entropy, forest__max_depth=16, grad__max_depth=8, grad__min_samples_split=28, forest__min_samples_split=211, total= 2.9min
[CV] grad__max_features=216, forest__max_featu

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 106.1min


[CV]  grad__max_features=317, forest__max_features=272, forest__criterion=entropy, forest__max_depth=310, grad__max_depth=6, grad__min_samples_split=255, forest__min_samples_split=38, total= 8.1min
[CV] grad__max_features=49, forest__max_features=60, forest__criterion=gini, forest__max_depth=302, grad__max_depth=6, grad__min_samples_split=318, forest__min_samples_split=220 
[CV]  grad__max_features=317, forest__max_features=272, forest__criterion=entropy, forest__max_depth=310, grad__max_depth=6, grad__min_samples_split=255, forest__min_samples_split=38, total= 8.1min
[CV] grad__max_features=49, forest__max_features=60, forest__criterion=gini, forest__max_depth=302, grad__max_depth=6, grad__min_samples_split=318, forest__min_samples_split=220 
[CV]  grad__max_features=27, forest__max_features=138, forest__criterion=gini, forest__max_depth=98, grad__max_depth=6, grad__min_samples_split=456, forest__min_samples_split=349, total= 1.2min
[CV] grad__max_features=216, forest__max_features=80

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 134.2min finished


{'grad__max_features': 128, 'forest__max_features': 132, 'forest__criterion': 'gini', 'forest__max_depth': 205, 'grad__max_depth': 9, 'grad__min_samples_split': 34, 'forest__min_samples_split': 243}


In [107]:
grid_result.best_params_

{'forest__criterion': 'gini',
 'forest__max_depth': 205,
 'forest__max_features': 132,
 'forest__min_samples_split': 243,
 'grad__max_depth': 9,
 'grad__max_features': 128,
 'grad__min_samples_split': 34}

In [106]:
grid_result.best_score_

0.78265575950639432

In [135]:
forest_final = RandomForestClassifier(n_jobs=-1, n_estimators=150, criterion='gini', max_depth=205, max_features=132, min_samples_split=243,  verbose=1)
grad_final = GradientBoostingClassifier(n_estimators=300, max_depth=9, max_features=128, min_samples_split=34,  verbose=1)
ada = AdaBoostClassifier(n_estimators=250)

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
eclf_final_test = VotingClassifier(estimators=[('forest', forest_final), ('grad', grad_final), ('ada', ada)], voting='soft', n_jobs=-1)
eclf_final_test.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.0939           83.61m


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.9s


         2           1.0599           80.70m
         3           1.0309           79.04m
         4           1.0062           78.00m
         5           0.9847           74.78m


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.4min finished


         6           0.9655           67.10m
         7           0.9477           60.57m
         8           0.9325           55.52m
         9           0.9193           51.98m
        10           0.9070           49.06m
        20           0.8221           33.43m
        30           0.7758           27.08m
        40           0.7392           22.96m
        50           0.7118           19.78m
        60           0.6885           17.49m
        70           0.6735           15.43m
        80           0.6610           13.75m
        90           0.6493           12.38m
       100           0.6394           11.24m
       200           0.5483            4.26m
       300           0.4711            0.00s


VotingClassifier(estimators=[('forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=205, max_features=132, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=243, min_weight_fraction_leaf=0.0,
            ...='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=250, random_state=None))],
         n_jobs=-1, voting='soft', weights=None)

In [137]:
eclf_final_test.score(X_test, y_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 150 out of 150 | elapsed:    0.4s finished


0.78367094479666

In [138]:
eclf_final = VotingClassifier(estimators=[('forest', forest_final), ('grad', grad_final), ('ada', ada)], voting='soft', n_jobs=-1)
eclf_final.fit(X, y)

      Iter       Train Loss   Remaining Time 
         1           1.0952           80.49m


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   23.1s


         2           1.0617           80.56m
         3           1.0354           76.08m
         4           1.0115           74.90m
         5           0.9909           74.50m


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.4min finished


         6           0.9736           69.11m
         7           0.9568           63.10m
         8           0.9429           58.24m
         9           0.9298           54.72m
        10           0.9187           51.49m
        20           0.8455           35.85m
        30           0.8037           28.37m
        40           0.7744           23.85m
        50           0.7527           20.58m
        60           0.7352           18.01m
        70           0.7217           15.97m
        80           0.7071           14.36m
        90           0.6971           12.89m
       100           0.6872           11.64m
       200           0.6144            4.28m
       300           0.5430            0.00s


VotingClassifier(estimators=[('forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=205, max_features=132, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=243, min_weight_fraction_leaf=0.0,
            ...='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=250, random_state=None))],
         n_jobs=-1, voting='soft', weights=None)

In [139]:
filename = 'final.sav'
joblib.dump(eclf_final, filename)

['final.sav']

In [143]:
import pandas as pd
y_pred = eclf_final.predict(X_test_input)
predictions = pd.DataFrame(X_test.ix[:,0])
predictions['PES1'] = pd.Series(y_pred)
predictions.to_csv(path_or_buf='output.txt',index=False)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 150 out of 150 | elapsed:    0.3s finished
